In [ ]:
i = 90
#Setting up plot
def set_plot(data):
    fig, ax = plt.subplots()
    #Get ticker values 
    print(data)
    #(add_days_to_date(datetime.date.fromisoformat(date), PERIOD)).strftime('%Y-%m-%d')
    #Set plot for stocks
    ax.plot(data[0], data[1])
    ax.xaxis.set_major_locator(tckr.MultipleLocator(1))


#Show elements in VBox
def set_output(output, i, percentage, text):
    with output:
        #Output iteration
        print(i)
        #Output text
        print(percentage)
        print(text)
        plt.show()
            
            
#Button events
def on_click_next(change):
    global i
    if i < len(articles)-1:
        output.clear_output()
        i+=1
        data = get_adj_close(articles[i]['date'])
        percentage = (data[1][1]-data[1][0])/data[1][0]*100
        set_plot(data)
        set_output(output, i, percentage, articles[i]['text'])
            
def on_click_prev(change):
    global i
    if i > 0:
        output.clear_output()
        #plt.clf()
        i-=1
        data = get_adj_close(articles[i]['date'])
        percentage = (data[1][1]-data[1][0])/data[1][0]*100
        set_plot(data)
        set_output(output, i, percentage, articles[i]['text'])

        
label = widgets.Label('Delete if not important')
btn_next = widgets.Button(description='Next')
btn_prev = widgets.Button(description='Prev')

output = widgets.Output()
#Button events connection
btn_next.on_click(on_click_next)
btn_prev.on_click(on_click_prev)
vbox = widgets.VBox([label, btn_next, btn_prev, output])
data = get_adj_close(articles[90]['date'])
percentage = (data[1][1]-data[1][0])/data[1][0]*100
set_plot(data)
set_output(output, 90, percentage, articles[90]['text'])
vbox

[['2022-01-17', '2022-01-20'], [3560.3501, 3516.02]]


In [25]:
#imoex = yf.Ticker(imoex_ticker)
data = yf.download(tickers="IMOEX.ME",start='2021-10-06', end='2022-05-7')
data['Adj Close'].to_csv('IMOEX.csv')

[*********************100%***********************]  1 of 1 completed


In [1]:
!pip install yfinance
!pip install pandas
!pip install bs4
!pip install lxml
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 63 kB 2.6 MB/s  eta 0:00:01
     |████████████████████████████████| 6.9 MB 34.8 MB/s eta 0:00:01
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8487 sha256=9bbacc2c72e8805bb98c489b43704157c12f10ecf16e0edab439379a724f41d3
  Stored in directory: /root/.cache/pip/wheels/21/c9/66/b41c847de65c7985db52ec21d59996841598b8b0e93f2b9500
Successfully built multitasking
  Attempting uninstall: requests
    Found existing installation: requests 2.25.0
    Uninstalling requests-2.25.0:
      Successfully uninstalled requests-2.25.0
     |████████████████████████████████| 128 kB 14.8 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=b4bbe61e346288cacc9a1df9a8e2b5d5b517ce0e459142c761a37ccea462d1f1
  Stored in directory: /root/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
     |████████████████████████████████| 325 kB 10.9 MB/s 

     |████████████████████████████████| 158 kB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 35.2 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 30.3 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 7.7 MB/s  eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895252 sha256=632a36b58896068afe2270bb0e6bd22e1139b5e6c75eb47678111d216c575557
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.52.0
    Uninstalling tqdm-4.52.0:
      Successfully uninstalled tqdm-4.52.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes 

## Импорты

In [14]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.ticker as tckr
import datetime
import yfinance as yf
import csv
from bs4 import BeautifulSoup
import requests
import json as jsn
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from transformers import RobertaForMaskedLM,RobertaTokenizer
from torch.utils.data import Dataset
import re

## Константы

In [2]:
imoex_ticker = "IMOEX.ME"
START_TIME = "16:00"
END_TIME = "23:00"
PERIOD = 3

IMOEX_PATH = "/notebooks/diplom/IMOEX.csv"
ARTICLES_PATH = "/notebooks/diplom/Articles.csv"
TRAINING_DATA_PATH = "/notebooks/diplom/Training_Data.csv"

root_url = "https://www.rbc.ru/v10/ajax/get-news-by-filters/?category=economics&offset="

## Загрузка статей

In [4]:
date = ""

file = open(ARTICLES_PATH, "w", newline='')
writer = csv.DictWriter(file, ['title', 'date', 'time', 'text'])
writer.writeheader()
for i in range(52):
    if date == '2019-01-01':
        break
    print(i)
    url = root_url + str(i*20) + "&limit=20"
    response = requests.get(url)
    try:
        json = response.json()
    except jsn.JSONDecodeError:
        continue
    soup = BeautifulSoup(json["html"], 'lxml')
    links = soup.find_all('a', class_='item__link', href=True)
    for link in links:
        href = link.get('href')
        print(href)
        sub_response = requests.get(href)
        sub_soup = BeautifulSoup(sub_response.text, 'lxml')
        title = sub_soup.find('h1', class_='article__header__title-in').text
        timestamp = sub_soup.find('span', class_="article__header__date")['content'].split('T')
        date = timestamp[0]
        time = timestamp[1][:5]
        text = ''
        article = sub_soup.find_all('div', class_="article__text")
        for part in article:
            tickers = part.select(".article__ticker")
            pro_anonses = part.select(".pro-anons")
            images = part.select(".article__main-image")
            inline_items = part.select(".article__inline-item")
            for ticker in tickers: ticker.decompose()
            for pro_anons in pro_anonses: pro_anons.decompose()
            for image in images: image.decompose()
            for inline_item in inline_items: inline_item.decompose()
            text+=part.text
        text = text.replace("\n", " ")
        
        writer.writerow({'title': title, 'date': date, 'time': time, 'text': text})
file.close()

0
https://www.rbc.ru/economics/05/05/2022/6273ce4e9a794709c999dda3
https://www.rbc.ru/economics/05/05/2022/6273c90e9a794708144d2063
https://www.rbc.ru/economics/05/05/2022/6273add89a79478039ec1d74
https://www.rbc.ru/rbcfreenews/62739d6f9a794778f0b500b6
https://www.rbc.ru/economics/05/05/2022/627266659a7947138b2f0c5e
https://www.rbc.ru/rbcfreenews/627303699a794749b8116abc
https://www.rbc.ru/economics/05/05/2022/6272f8399a794746790b8afe
https://www.rbc.ru/economics/05/05/2022/62723c399a7947fc6c4d28a2
https://www.rbc.ru/economics/04/05/2022/627279659a79471c9b6a5a85
https://www.rbc.ru/rbcfreenews/627262b59a79471224bd5320
https://www.rbc.ru/economics/04/05/2022/627260a69a7947113f542ad3
https://www.rbc.ru/rbcfreenews/6271faa89a7947e496b643a2
https://www.rbc.ru/rbcfreenews/6271cf499a7947d7acd6da27
https://www.rbc.ru/rbcfreenews/6271b0a49a7947d29c39a6c8
https://www.rbc.ru/economics/03/05/2022/6271516d9a7947c0dd95bbac
https://www.rbc.ru/rbcfreenews/6270f45f9a7947abc9791a60
https://www.rbc.ru/ec

https://www.rbc.ru/economics/13/04/2022/62544eeb9a7947e9c1528d76
https://www.rbc.ru/economics/12/04/2022/6255941f9a79475a66ebde28
https://www.rbc.ru/economics/12/04/2022/62555a109a79473be3d198e1
https://www.rbc.ru/economics/12/04/2022/62552aed9a79472686a5092c
https://www.rbc.ru/economics/12/04/2022/62551eb99a79472174a63603
https://www.rbc.ru/economics/12/04/2022/6254ff0e9a794714c4361a68
https://www.rbc.ru/rbcfreenews/6254f9f59a79471297b10770
https://www.rbc.ru/economics/11/04/2022/62543f679a7947e285c71eb0
https://www.rbc.ru/economics/11/04/2022/625460419a7947ef99c4dcde
https://www.rbc.ru/economics/11/04/2022/625460539a7947ef99c4dce1
7
https://www.rbc.ru/economics/11/04/2022/625435289a7947d896b60d8a
https://www.rbc.ru/rbcfreenews/625380229a79478e181079da
https://www.rbc.ru/economics/11/04/2022/625366779a794785b32656a8
https://www.rbc.ru/economics/11/04/2022/625351f39a794782999ff6fa
https://www.rbc.ru/economics/11/04/2022/62533fb89a79477cfe6995be
https://www.rbc.ru/economics/11/04/2022/6

https://www.rbc.ru/economics/24/03/2022/623c3fbd9a794732ec4a4d58
https://www.rbc.ru/rbcfreenews/623c06089a794716697c3855
13
https://www.rbc.ru/economics/24/03/2022/623bc4259a794702d0046359
https://www.rbc.ru/economics/24/03/2022/623bb03a9a794703ed34fd63
https://www.rbc.ru/economics/24/03/2022/623b9ded9a794785c69f5279
https://www.rbc.ru/economics/23/03/2022/623b32609a794761bc20d5bb
https://www.rbc.ru/economics/23/03/2022/623b5bcc9a79477293c33b56
https://www.rbc.ru/economics/23/03/2022/623b65629a794775df1924f3
https://www.rbc.ru/economics/23/03/2022/623b1b729a79475646ea9d32
https://www.rbc.ru/economics/23/03/2022/623b2b159a79475e3c3ba0cc
https://www.rbc.ru/rbcfreenews/623ab1e39a7947204a9ca88a
https://www.rbc.ru/economics/23/03/2022/6234974b9a7947e6caa7235e
https://www.rbc.ru/rbcfreenews/6239ee369a7947edf82de567
https://www.rbc.ru/economics/22/03/2022/6239cb2a9a7947ddddda0c2c
https://www.rbc.ru/economics/22/03/2022/6238489b9a7947510923b77f
https://www.rbc.ru/economics/22/03/2022/6239468a9

https://www.rbc.ru/economics/09/03/2022/622870549a794756af1cb9e0
https://www.rbc.ru/economics/09/03/2022/622847c29a794744623e5412
https://www.rbc.ru/economics/09/03/2022/622837d89a79473dab87a7a9
https://www.rbc.ru/economics/09/03/2022/62233c079a7947025ae2a7b4
https://www.rbc.ru/economics/09/03/2022/6227e0369a79472a6ad64fe7
https://www.rbc.ru/economics/09/03/2022/6227d6db9a79472b37c3a07b
https://www.rbc.ru/economics/08/03/2022/622793ba9a7947180e01950c
https://www.rbc.ru/economics/08/03/2022/62279e029a79471a285c5f5c
https://www.rbc.ru/rbcfreenews/62277a469a794710f91b2385
https://www.rbc.ru/economics/08/03/2022/6227845c9a79471320255989
https://www.rbc.ru/economics/08/03/2022/62276ccf9a79470edab85203
https://www.rbc.ru/economics/08/03/2022/622760169a79470b5277dacc
https://www.rbc.ru/economics/08/03/2022/62275ba59a794709168c5fa4
https://www.rbc.ru/economics/08/03/2022/62273f319a79470298b326bf
20
https://www.rbc.ru/economics/08/03/2022/622733d09a7947005134b656
https://www.rbc.ru/economics/08

https://www.rbc.ru/economics/15/02/2022/620637509a7947565f455f87
https://www.rbc.ru/economics/15/02/2022/620a80779a79471c10a6d887
https://www.rbc.ru/economics/14/02/2022/620ab3c59a79472d7b5a14a8
https://www.rbc.ru/economics/14/02/2022/620a86819a79471e6c049582
https://www.rbc.ru/economics/14/02/2022/620a25989a79476ea4e7f3b3
26
https://www.rbc.ru/economics/14/02/2022/6206961d9a7947868650e9c7
https://www.rbc.ru/economics/14/02/2022/6209b5339a794747aaaf4a63
https://www.rbc.ru/economics/14/02/2022/620663319a79476ea1ba7196
https://www.rbc.ru/rbcfreenews/62088d019a794700a920847d
https://www.rbc.ru/rbcfreenews/620667269a79476fccffdb15
https://www.rbc.ru/rbcfreenews/62063d049a7947591ece0515
https://www.rbc.ru/economics/11/02/2022/6205fbc29a794736b2960e19
https://www.rbc.ru/economics/11/02/2022/6205e76c9a79472edafbb22f
https://www.rbc.ru/economics/11/02/2022/620531b19a7947861d59cf1d
https://www.rbc.ru/economics/11/02/2022/62055f659a79471065483290
https://www.rbc.ru/economics/10/02/2022/62054c679

https://www.rbc.ru/economics/06/01/2022/61d5ce179a794744b358e54f
https://www.rbc.ru/economics/06/01/2022/61ca21ef9a79474ff7c6aef4
https://www.rbc.ru/economics/05/01/2022/61d54d2a9a794721c8c5a69b
https://www.rbc.ru/economics/05/01/2022/61cd7a909a79476b509fd082
https://www.rbc.ru/economics/04/01/2022/61d48bdf9a79477ce0a4cf5a
https://www.rbc.ru/economics/04/01/2022/61d4685d9a794775b6c54ae3
https://www.rbc.ru/economics/04/01/2022/61d420e19a794764f06b438b
https://www.rbc.ru/economics/04/01/2022/61d401e59a79475bcb0f16cd
https://www.rbc.ru/economics/04/01/2022/61c5dea69a7947796ab06e1c
https://www.rbc.ru/economics/03/01/2022/61d2a8f49a79470e106e5b29
https://www.rbc.ru/economics/03/01/2022/61c1ac739a79476c95bd7775
https://www.rbc.ru/economics/03/01/2022/61d246289a7947f84247e910
https://www.rbc.ru/economics/02/01/2022/61d144b09a7947c07df93c28
33
https://www.rbc.ru/economics/01/01/2022/61d076059a79479bc2463e3e
https://www.rbc.ru/economics/01/01/2022/61c07e579a794700d6027612
https://www.rbc.ru/eco

https://www.rbc.ru/economics/05/12/2021/61ac4e739a79475bcfcdc65a
39
https://www.rbc.ru/rbcfreenews/61aba59e9a79473a59776369
https://www.rbc.ru/rbcfreenews/61aaf0689a79470e6d25596a
https://www.rbc.ru/economics/04/12/2021/61aaaaee9a7947013236a83e
https://www.rbc.ru/economics/04/12/2021/61aa984a9a7947fe4670a2de
https://www.rbc.ru/economics/03/12/2021/61aa2b239a7947deedb82d37
https://www.rbc.ru/economics/03/12/2021/61aa34739a7947e24f034e11
https://www.rbc.ru/economics/03/12/2021/61aa3f909a7947e737b0e5bc
https://www.rbc.ru/economics/03/12/2021/61a91e299a7947842e4918f9
https://www.rbc.ru/economics/03/12/2021/61a8d2789a794767a52cd1a2
https://www.rbc.ru/rbcfreenews/61a8a4c09a7947531e733ddf
https://www.rbc.ru/economics/02/12/2021/61a7f9fd9a794721b1655abd
https://www.rbc.ru/economics/01/12/2021/61a78a8b9a794784b3ba26f6
https://www.rbc.ru/rbcfreenews/61a781129a794781321dd942
https://www.rbc.ru/rbcfreenews/61a7825f9a79478183743842
https://www.rbc.ru/economics/01/12/2021/61a62c7a9a794707ceef3e33
ht

https://www.rbc.ru/economics/31/10/2021/617e0f339a7947799213aa89
https://www.rbc.ru/rbcfreenews/617d38fc9a79474c02af64ac
https://www.rbc.ru/economics/30/10/2021/617d05bf9a7947404ac4a81a
https://www.rbc.ru/rbcfreenews/617cf39b9a79473b1b08c780
https://www.rbc.ru/economics/30/10/2021/617c2cff9a794711deed8488
https://www.rbc.ru/rbcfreenews/617c25e39a79470fb0d5c3fa
https://www.rbc.ru/economics/29/10/2021/617c19909a794709f19d508f
https://www.rbc.ru/economics/29/10/2021/617c051b9a794701172a3a83
46
https://www.rbc.ru/economics/29/10/2021/617c0fa89a7947063cac6ed0
https://www.rbc.ru/economics/29/10/2021/617bec699a7947f69e9f7935
https://www.rbc.ru/rbcfreenews/617be45c9a7947f1b85c232b
https://www.rbc.ru/rbcfreenews/617ba99d9a7947d476c3b5e9
https://www.rbc.ru/rbcfreenews/617b962c9a7947cc50427dfb
https://www.rbc.ru/economics/28/10/2021/617b05cf9a7947abce3a10e4
https://www.rbc.ru/economics/28/10/2021/617a81319a7947761f037fa7
https://www.rbc.ru/rbcfreenews/617a334a9a79475319c40d10
https://www.rbc.ru/e

## Вытаскиваем инфу из файлов

In [59]:
class CorpusExtractor():
    def __init__(self, data_path, label_path):
        print("getting articles")
        self.articles = self.get_articles(data_path)
        print("getting text of articles")
        self.texts = self.get_texts()
        print("getting labels")
        self.labels = self.get_labels(label_path)
    
    # getting articles from csv file
    def get_articles(self, data_path):
        articles = []
        with open(data_path, "r", newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                articles.append(row)
        return articles
    
    # getting articles' text from variavble 'articles'
    def get_texts(self):
        return [self.preprocess_text(article["text"], i) for i, article in enumerate(self.articles)]

    # getting articles' labels from variavble 'articles'
    def get_labels(self, label_path):
        labels = []
        for i, article in enumerate(self.articles):
            print(str(i) + "/1019")
            date = article["date"]
            indexes = self.get_adj_close(date, label_path)
            while not indexes and datetime.date.fromisoformat(date) <= datetime.datetime.now().date():
                date = self.add_days_to_date(date, 1)
                indexes = self.get_adj_close(date, label_path)
            labels.append(self.get_percenatge(indexes))
        return labels
    
    # preprocessing text
    def preprocess_text(self, text, i):
        print(str(i) + "/1019")
        text = text.lower().replace("ё", "е")
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
        text = re.sub('@[^\s]+', 'USER', text)
        text = re.sub('[^a-zA-Zа-яА-Я0-9]+', ' ', text)
        text = re.sub(' +', ' ', text)
        return text.strip()

    # getting index values for the period
    def get_adj_close(self, start, label_path, n = PERIOD):
        dates, indexes = [], []
        row_id = 0
        with open(label_path, "r") as file:
            reader = csv.DictReader(file)
            for i, row in enumerate(reader):
                if row["Date"] == start:
                    row_id = i
                    dates.append(start)
                    indexes.append(round(float(row["Adj Close"]), 4))
                if dates and i - row_id == n:
                    dates.append(row["Date"])
                    indexes.append(round(float(row["Adj Close"]), 4))
        if len(indexes) == 2:
            return indexes
        else: return []
    
    # counting the difference precentage
    def get_percenatge(self, indexes):
        if indexes:
            return round((indexes[1] - indexes[0])/indexes[0]*100, 4)
        return
    
    # Add n-days to data
    def add_days_to_date(self, date_str, n_days):
        new_date = datetime.date.fromisoformat(date_str) + datetime.timedelta(days=n_days)
        return new_date.strftime('%Y-%m-%d')
    
    # saving the data
    def save_data(self, save_path):
        with open(save_path, "w", newline='') as file:
            writer = csv.DictWriter(file, ['text', 'label'])
            writer.writeheader()
            for i in range(len(self.texts)):
                writer.writerow({'text': self.texts[i], 'label': self.labels[i]})

In [60]:
ce = CorpusExtractor(ARTICLES_PATH, IMOEX_PATH)
ce.save_data(TRAINING_DATA_PATH)

getting articles
getting text of articles
0/1019
1/1019
2/1019
3/1019
4/1019
5/1019
6/1019
7/1019
8/1019
9/1019
10/1019
11/1019
12/1019
13/1019
14/1019
15/1019
16/1019
17/1019
18/1019
19/1019
20/1019
21/1019
22/1019
23/1019
24/1019
25/1019
26/1019
27/1019
28/1019
29/1019
30/1019
31/1019
32/1019
33/1019
34/1019
35/1019
36/1019
37/1019
38/1019
39/1019
40/1019
41/1019
42/1019
43/1019
44/1019
45/1019
46/1019
47/1019
48/1019
49/1019
50/1019
51/1019
52/1019
53/1019
54/1019
55/1019
56/1019
57/1019
58/1019
59/1019
60/1019
61/1019
62/1019
63/1019
64/1019
65/1019
66/1019
67/1019
68/1019
69/1019
70/1019
71/1019
72/1019
73/1019
74/1019
75/1019
76/1019
77/1019
78/1019
79/1019
80/1019
81/1019
82/1019
83/1019
84/1019
85/1019
86/1019
87/1019
88/1019
89/1019
90/1019
91/1019
92/1019
93/1019
94/1019
95/1019
96/1019
97/1019
98/1019
99/1019
100/1019
101/1019
102/1019
103/1019
104/1019
105/1019
106/1019
107/1019
108/1019
109/1019
110/1019
111/1019
112/1019
113/1019
114/1019
115/1019
116/1019
117/1019
118/10

1/1019
2/1019
3/1019
4/1019
5/1019
6/1019
7/1019
8/1019
9/1019
10/1019
11/1019
12/1019
13/1019
14/1019
15/1019
16/1019
17/1019
18/1019
19/1019
20/1019
21/1019
22/1019
23/1019
24/1019
25/1019
26/1019
27/1019
28/1019
29/1019
30/1019
31/1019
32/1019
33/1019
34/1019
35/1019
36/1019
37/1019
38/1019
39/1019
40/1019
41/1019
42/1019
43/1019
44/1019
45/1019
46/1019
47/1019
48/1019
49/1019
50/1019
51/1019
52/1019
53/1019
54/1019
55/1019
56/1019
57/1019
58/1019
59/1019
60/1019
61/1019
62/1019
63/1019
64/1019
65/1019
66/1019
67/1019
68/1019
69/1019
70/1019
71/1019
72/1019
73/1019
74/1019
75/1019
76/1019
77/1019
78/1019
79/1019
80/1019
81/1019
82/1019
83/1019
84/1019
85/1019
86/1019
87/1019
88/1019
89/1019
90/1019
91/1019
92/1019
93/1019
94/1019
95/1019
96/1019
97/1019
98/1019
99/1019
100/1019
101/1019
102/1019
103/1019
104/1019
105/1019
106/1019
107/1019
108/1019
109/1019
110/1019
111/1019
112/1019
113/1019
114/1019
115/1019
116/1019
117/1019
118/1019
119/1019
120/1019
121/1019
122/1019
123/1019
1

## Формируем dataset из полученного корпуса 

In [61]:
# downloading pre-trained RoBERTa model for russian language
model=RobertaForMaskedLM.from_pretrained('sberbank-ai/ruRoberta-large')
tokenizer=RobertaTokenizer.from_pretrained('sberbank-ai/ruRoberta-large')

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [63]:
class CustomDataset(Dataset):
    def __init__(self, data_path, exract_features = False):
        df = pd.read_csv(data_path)
        self.texts = df['text'].to_list()
        self.labels = df['label'].to_list()
        self.exract_features = exract_features
        if self.exract_features:
            self.texts = get_features()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label
    
    def get_features(self):
        tokenized = tokenizer(self.texts, add_special_tokens=True, return_tensors='pt', padding=True)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
        with torch.no_grad():
            last_hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)
        features = last_hidden_states.logits[:,0,:]
        return features
    
ds = CustomDataset(TRAINING_DATA_PATH)
# df['text'] = df['text'].str.slice(0,514)

In [172]:
# ruRoberta-large example 

#tokenized = tokenizer('Снижение цен в Москве возможно только при.', return_tensors='np')
tokenized = tokenizer(df.text[:10].to_list(), add_special_tokens=True, return_tensors='pt', padding=True)

input_ids = tokenized.input_ids
attention_mask = tokenized.attention_mask

with torch.no_grad():
    last_hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)
features = last_hidden_states.logits[:,0,:]

print(features.shape)

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided []

In [62]:
labels = df['label'][:10]

NameError: name 'df' is not defined

In [ ]:
ds = CustomDataset(features, labels)
for epoch in range(3):
    